# EDA #
The script has gathered 100 reviews per user for almost 300 users. Time to make sure all the data is present

In [1]:
import pandas as pd
import sqlite3
import os
from pathlib import Path

In [2]:
# File path to the consolidated data
path = '../data/interim/consolidated_data.csv'

In [3]:
df = pd.read_csv(path)
df.head()

,title,author,avg_rating,user_rating,cover_url,user_id
0,Harry Potter and the Sorcerer's Stone (Harry P...,NaN,4.47,5,https://i.gr-assets.com/images/S/compressed.ph...,1
1,"The Hunger Games (The Hunger Games, #1)",NaN,4.33,5,https://i.gr-assets.com/images/S/compressed.ph...,1
2,To Kill a Mockingbird,NaN,4.26,5,https://i.gr-assets.com/images/S/compressed.ph...,1
3,The Great Gatsby,NaN,3.93,4,https://i.gr-assets.com/images/S/compressed.ph...,1
4,1984,NaN,4.19,5,https://i.gr-assets.com/images/S/compressed.ph...,1


In [4]:
df.shape

(29530, 6)

In [5]:
df['title'].nunique()

6334

In [6]:
df['cover_url'].nunique()

9872

In [7]:
df.isna().sum()

title              0
author         29530
avg_rating         0
user_rating        0
cover_url          0
user_id            0
dtype: int64

In [8]:
df.user_id.unique().dtype

dtype('int64')

In [9]:
non_integers = df[pd.to_numeric(df['user_id'], errors='coerce').isna()]
print(non_integers)

Empty DataFrame
Columns: [title, author, avg_rating, user_rating, cover_url, user_id]
Index: []


### First Impressions ###
There are 29,530 reviews here, but only 6,334 unique books. This isn't a huge surprise, since I sorted each user's "read" shelf by most-reviewed and scraped the top 100. I'm actually glad to see this much variety. This should mean that I have legitimate recommendation power but that I don't recommend very obscure or unpopular books. There are more unique cover URLs due to different editions existing for many of the books.

The script also failed to gather the "author" column for some reason. This information would be nice to have but ultimately isn't worth the time to re-run the script. If this project were going into production, I'd gather far more than title, author, and rating anyways. I'll drop the author column for now.

In [10]:
try:
    df = df.drop('author', axis = 1)
except:
    pass
df.head()

,title,avg_rating,user_rating,cover_url,user_id
0,Harry Potter and the Sorcerer's Stone (Harry P...,4.47,5,https://i.gr-assets.com/images/S/compressed.ph...,1
1,"The Hunger Games (The Hunger Games, #1)",4.33,5,https://i.gr-assets.com/images/S/compressed.ph...,1
2,To Kill a Mockingbird,4.26,5,https://i.gr-assets.com/images/S/compressed.ph...,1
3,The Great Gatsby,3.93,4,https://i.gr-assets.com/images/S/compressed.ph...,1
4,1984,4.19,5,https://i.gr-assets.com/images/S/compressed.ph...,1


# SQL Database Exploration #
Now I want to check out the relational database I made from the data

In [11]:
database_path = os.path.abspath(
            os.path.join(
                os.pardir,
                'data',
                'processed',
                'books.db'
                )
            )

conn = sqlite3.connect(database_path)

In [12]:
cur = conn.cursor()
cur.execute('SELECT * FROM Ratings LIMIT 10')
results = cur.fetchall()

for row in results:
    print(row)

(1, 1, 5, 4.47)
(10160415, 1, 0, 4.47)
(1019021, 1, 5, 4.47)
(10367112, 1, 4, 4.47)
(104823, 1, 4, 4.47)
(1053760, 1, 5, 4.47)
(10553671, 1, 5, 4.47)
(106638288, 1, 3, 4.47)
(1072582, 1, 5, 4.47)
(10915830, 1, 5, 4.47)


In [13]:
cur = conn.cursor()
cur.execute('SELECT * FROM Books LIMIT 10')
results = cur.fetchall()

for row in results:
    print(row)

(1, "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)", 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1474154022l/3._SY75_.jpg')
(2, 'The Hunger Games (The Hunger Games, #1)', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1586722975l/2767052._SX50_.jpg')
(3, 'To Kill a Mockingbird', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1553383690l/2657._SY75_.jpg')
(4, 'The Great Gatsby', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1490528560l/4671._SY75_.jpg')
(5, '1984', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1348990566l/5470._SY75_.jpg')
(6, 'Pride and Prejudice', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1622158493l/1893._SX50_.jpg')
(7, 'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)', 'https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1630547330l/5._SY75_.jpg')
(8, 'The Hobbit (The Lord 

In [14]:
cur.execute('SELECT COUNT(*) FROM Books')
book_count = cur.fetchone()
print(book_count)

(9909,)


In [15]:
cur.execute('SELECT COUNT(*) FROM Users')
user_count = cur.fetchone()
print(user_count)

(298,)


In [16]:
for row in conn.execute("PRAGMA table_info(Ratings)"):
    print(row)

(0, 'user_id', 'INTEGER', 0, None, 0)
(1, 'book_id', 'INTEGER', 0, None, 0)
(2, 'user_rating', 'INTEGER', 0, None, 0)
(3, 'avg_rating', 'REAL', 0, None, 0)


In [17]:
conn.close()

## Results ##
The database is correctly built with all books, users, and ratings accounted for! There are 9902 unique combinations of book title and cover URL. While I'd like to go through and merge all unique editions into a single entry, I don't expect it would be worth the effort for the purposes of this project.